*** This practice is based on the tutorial (5-Day Data Challege Day one: Data Cleaning Challenge: Handling missing values) given by Rachael Tatman on Kaggle ***
  

In [1]:
%autosave 9999

Autosaving every 9999 seconds


In [2]:
import pandas as pd
import numpy as np
nfl_data = pd.read_csv("../data/NFL Play by Play 2009-2017 (v4).csv")
sf_permits = pd.read_csv("../data/Building_Permits.csv")
# set seed for reproducibility
np.random.seed(0) 

/Users/yanglu/anaconda3/envs/env1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/yanglu/anaconda3/envs/env1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


*** The first thing we need to do is just have a glance of the data ***

In [3]:
#nfl_data.info() #show the overall information of the dataset
#nfl_data.describe() # show statistic informtion of the dataset including mean, std, etc.
#nfl_data.head(5)  #show the first 5 rows of the dataset 
#nfl_data.tail(5)  # show the last 5 rows of the dataset
nfl_data.sample(5) # return 5 random samples of the dataset

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
244485,2014-10-26,2014102607,18,3,1.0,00:39,1,939.0,12.0,TB,...,1.240299,0.225647,0.774353,0.245582,0.754418,0.225647,0.019935,-0.018156,0.038091,2014
115340,2011-11-20,2011112000,22,4,1.0,06:47,7,407.0,44.0,OAK,...,NaN,0.056036,0.943964,0.042963,0.957037,0.943964,0.013073,NaN,NaN,2011
68357,2010-11-14,2010111401,8,2,NaN,00:23,1,1823.0,0.0,CLE,...,NaN,0.365307,0.634693,0.384697,0.615303,0.634693,-0.019390,NaN,NaN,2010
368377,2017-09-24,2017092405,24,4,1.0,08:48,9,528.0,8.0,CLE,...,1.075660,0.935995,0.064005,0.921231,0.078769,0.064005,0.014764,0.003866,0.010899,2017
384684,2017-11-05,2017110505,11,2,1.0,09:15,10,2355.0,0.0,DEN,...,NaN,0.928474,0.071526,0.934641,0.065359,0.071526,-0.006166,NaN,NaN,2017


*** If there is missing data, i.e. NaN, we need to count the number of them. ***

In [4]:
missing_values_count = nfl_data.isnull().sum() #count the missing data in each column
missing_values_count[0:10] # show the corresponding information of the first 10 columns

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

*** Calculate the missing data percentage ***

In [5]:
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()
(total_missing/total_cells)* 100

24.87214126835169

# Figure out why the data is missing
____
 
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it: 

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

*** Drop missing values ***
If it is not time limited, this method is not recommanded

In [6]:
nfl_data.dropna() # drop rows which contain missing values

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


In [7]:
columns_with_na_dropped = nfl_data.dropna(axis=1) # drop columns which contains missing values
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 102 

Columns with na's dropped: 41


*** Filling in missing data ***

In [8]:
# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


In [12]:
#subset_nfl_data.fillna(0) # fill in with 0
#fill in with the value that comes directly after it in the same column, last row fill in with 0 
#subset_nfl_data.fillna(method = 'bfill', axis=0).fillna("0") 
#subset_nfl_data.fillna(nfl_data.mean()) # fill in with mean value of each column. It may take miniutes when the dataset is large
#subset_nfl_data.fillna(nfl_data.max()) #fill in with the maximum value
#subset_nfl_data.fillna(set_nfl_data.min()) #fill in with the minimum value

subset_nfl_data.fillna(subset_nfl_data.mean()) #to save time just use the subset

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,1.125336,-1.942674,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.037209,-0.059670,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,1.125336,-1.942674,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.037209,-0.059670,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,1.125336,-1.942674,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.037209,-0.059670,2009


*** Try Imputer via sklearn ***
https://www.kaggle.com/dansbecker/handling-missing-values

In [18]:
from sklearn.preprocessing import Imputer
my_imputer = Imputer() #default method is filling in with mean value
data_with_imputed_values = my_imputer.fit_transform(subset_nfl_data) # the returned value is an array
# transfer back to the Dataframe
data_with_imputed_values_pd = pd.DataFrame(data_with_imputed_values,index=subset_nfl_data.index,columns=subset_nfl_data.columns)
data_with_imputed_values_pd

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,1.125336,-1.942674,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.037209,-0.059670,2009.0
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009.0
2,-1.402760,1.125336,-1.942674,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.037209,-0.059670,2009.0
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009.0
4,2.097796,1.125336,-1.942674,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.037209,-0.059670,2009.0
